In [17]:
from importlib import reload
import audioDatastore.AudioDatastore as myads
reload(myads)
import os.path
from collections import Counter
import random
from python_speech_features import mfcc, delta
import scipy.io.wavfile as wav
from spafe.utils import vis
import matplotlib.pyplot as plt
import misc.vad as vad
from sklearn.mixture import GaussianMixture, _gaussian_mixture
reload(vad)
import numpy as np
from scipy.special import logsumexp
from collections import Counter
import helper_functions as hf
reload(hf)
import pickle

In [12]:
# create ads
# female ads
datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\SPEECH DATA\FEMALE\MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\SPEECH DATA\MALE\MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels+male_ads.labels)

In [13]:
# split ads
developmentLabels = ["M01","M02","M03","M04","M06","M07","M08","M09","F01","F02","F03","F04","F06","F07","F08","F09"]
evaluationLabels = ["M05","M10","F05","F10"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
numFilesPerSpeakerForEnrollment = 3
adsEnroll, adsDet = myads.split(adsEvaluate, 3)
adsTest, adsDet = myads.split(adsDet, 2)

In [14]:
speedUpExamples = True
if speedUpExamples:
    adsTrain, _ = myads.split(adsTrain, 30)
    adsDet, _ = myads.split(adsDet, 21)

In [15]:
# set sample rate,
fs, _ = wav.read(adsTrain.files[0])

In [70]:
reload = False

try:
    if reload :
        raise Exception('reloading')
    training_features = pickle.load(open('training_features.pickle','rb'))
except:
    training_features = []
    for i in range(len(adsTrain.files)):
        if i%50 == 0:
            print('completed ' + str(i) + ' iterations')
        feat = hf.helper_feature_extraction(adsTrain.files[i])
        training_features.append(feat)
    print('completed training feature fe!')
    pickle.dump(training_features, open('training_features.pickle','wb'))

In [22]:
# collect normalization factors

import numpy as np

means = []
std = []
for feature in training_features:
    if len(feature) > 0:
        means.append(np.mean(feature, axis=0))
        std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.mean(std, axis=0)

class NormFactor:
  def __init__(self, m, s):
    self.means = m
    self.std = s

normFactors = NormFactor(means, std)

try:
    if reload:
        raise Exception('reloading')
    normFactors = pickle.load(open('normFactors.pickle','rb'))
except:
    pickle.dump(normFactors, open('normFactors.pickle','wb'))

In [25]:
normalised_features = []

try:
    normalised_features = pickle.load(open('normalised_features.pickle','rb'))
    if True:
        raise Exception('reloading')
except:
    for i in range(len(training_features)):
        if len(training_features[i]) > 0:
            normalised = (training_features[i] - means) / std
            normalised = normalised - np.mean(normalised)
            normalised_features.append(normalised)
        else:
            normalised_features.append(np.array([]))
    pickle.dump(normalised_features, open('normalised_features.pickle','wb'))
    print('saved normalised features!')

saved normalised features!


In [27]:
numComponents = 32
ubm = GaussianMixture(n_components=numComponents, covariance_type='diag')
train_features_flattened = np.array([item for sublist in training_features for item in sublist])
ubm.fit(train_features_flattened)
print('sklearn model' + str(ubm.score(training_features[0])))
pickle.dump(ubm, open('ubm.pickle','wb'))

sklearn model-22.570612822365575


In [ ]:
# compute BW stats